# Title Here

author1,author2,author3,author4

# Research Background

The foreign exchange (FX) market is one of the world's largest financial markets, with daily trading volumes that can reach trillions of dollars.

The carry trade is a popular trading strategy in the FX market, from where the investors invest in high-interest currencies by borrowing in low-interest currencies with the purpose of earning the interest rate differential. 

## Research Target

Based on the methodologies of quantitative time series analysis, we reserach the TRI series of AUD-USD currency pair, aim to fitting a significant model with historical data and getting solid predictions of future.

## Carry trade

In carry trade, we always hope to with "positive carry", i.e. "borrow" in low-interest currency and "lend" in high-interest currency.

![carry](../img/carry.png)

## TRI series

Our research is based on the daily TRI series of the AUD-USD currency pair from 1997 to 2022. TRI means the total return index of carry trade, which equals to the sum of spot change and cumulative carry. 

$$
TRI_{t} = SPOT_{t} + \sum_{i=1}^{t}CARRY_{i}
$$

<img src="../img/tri.png" style="zoom:.8">

| **timestamp**       | **tri**   | **spot**  | **carry** | **log_return** |
| ------------------- | --------- | --------- | --------- | -------------- |
| 1997-03-20 19:00:00 | 50.090012 | 0.089139  | 0.224690  | 0.001799       |
| 1997-03-21 19:00:00 | 49.827164 | -0.174776 | 0.198000  | -0.005261      |
| 1997-03-24 19:00:00 | 50.163459 | 0.160175  | 0.207871  | 0.006727       |
| 1997-03-25 19:00:00 | 50.133008 | 0.129161  | 0.198581  | -0.000607      |
| 1997-03-26 19:00:00 | 49.511216 | -0.500142 | 0.254483  | -0.012480      |